### Initialize notebook

In [11]:
# Torch imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Other imports
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir('/home/ak/Spring2018/ature')
sep = os.sep

# Define folders (create them if needed)
Dirs = {}
Dirs['data']      = 'data'+sep+'DRIVE'+sep+'test'+sep +'patches'

# Set up execution flags
Flags = {}
Flags['useGPU'] = False

### Retrive the CIFAR 10 images if needed

In [29]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

print('Data folder is: ' + Dirs['data'])

arr =np.load(Dirs['data']+sep+'16_test.npy')
data_tensor = torch.IntTensor(arr[:, 0:2601])
y_tensor = torch.IntTensor(arr[:,2601])
trainset = torch.utils.data.TensorDataset(data_tensor, y_tensor)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=0)

# testset = torchvision.datasets.CIFAR10(root=Dirs['data'], train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4,
#                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Data folder is: data/DRIVE/test/patches


RuntimeError: $ Torch: not enough memory: you tried to allocate 1GB. Buy new RAM! at /pytorch/torch/lib/TH/THGeneral.c:253

In [27]:
for i,j in enumerate(trainloader,0):
    print(j)
    break

OSError: [Errno 12] Cannot allocate memory

### Define the network

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.sm = nn.LogSoftmax()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sm(x)
        return x

net = Net()

# Send network to the GPU, if requested
if Flags['useGPU']:
    net.cuda()

# Define loss criterion
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


### Train the network

In [5]:
for epoch in range(4):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        if Flags['useGPU']:
            # Wrap them in Variable (CPU version)
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())                
        else:                
            # Wrap them in Variable (GPU version)
            inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


/home/ak/Spring2018/ature_env/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1,  2000] loss: 2.183
[1,  4000] loss: 1.847
[1,  6000] loss: 1.666
[1,  8000] loss: 1.581
[1, 10000] loss: 1.518
[1, 12000] loss: 1.467
[2,  2000] loss: 1.388
[2,  4000] loss: 1.364
[2,  6000] loss: 1.335
[2,  8000] loss: 1.313
[2, 10000] loss: 1.283
[2, 12000] loss: 1.261
[3,  2000] loss: 1.209
[3,  4000] loss: 1.177
[3,  6000] loss: 1.201
[3,  8000] loss: 1.176
[3, 10000] loss: 1.161
[3, 12000] loss: 1.163
[4,  2000] loss: 1.092
[4,  4000] loss: 1.084
[4,  6000] loss: 1.107
[4,  8000] loss: 1.098
[4, 10000] loss: 1.094
[4, 12000] loss: 1.083
Finished Training


### Test the network

In [6]:
correct = 0
total = 0
for data in testloader:
    images, labels = data        
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


/home/ak/Spring2018/ature_env/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 59 %


### Check per-class performance

In [7]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


/home/ak/Spring2018/ature_env/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of plane : 62 %
Accuracy of   car : 72 %
Accuracy of  bird : 55 %
Accuracy of   cat : 33 %
Accuracy of  deer : 46 %
Accuracy of   dog : 57 %
Accuracy of  frog : 62 %
Accuracy of horse : 61 %
Accuracy of  ship : 69 %
Accuracy of truck : 70 %


In [8]:
trainloader

In [18]:
trainset = torch.utils.data.TensorDataset([0,1],[0,1])

AttributeError: 'list' object has no attribute 'size'